In [1]:
!git clone https://github.com/Js-Hwang1/LST.git

Cloning into 'LST'...
remote: Enumerating objects: 859, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 859 (delta 18), reused 32 (delta 12), pack-reused 816 (from 1)
Receiving objects: 100% (859/859), 387.13 MiB | 41.52 MiB/s, done.
Resolving deltas: 100% (408/408), done.
Updating files: 100% (122/122), done.


In [2]:
!nvidia-smi

Sat Dec 27 06:12:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             41W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
!ls

LST  sample_data


In [5]:
import os
os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN_HERE"

!cd LST && python scripts/benchmark/eval_longbench.py \
    --model_name mistralai/Mistral-7B-Instruct-v0.2 \
    --methods dense \
    --tasks qasper,multifieldqa_en,hotpotqa,2wikimqa,musique,gov_report,multi_news,trec,triviaqa,samsum,lcc,repobench-p \
    --num_samples 100 \
    --output_file results/longbench_mistral_dense.json



2025-12-27 06:12:39,137 - INFO - Using device: cuda
2025-12-27 06:12:47,324 - INFO - NumExpr defaulting to 12 threads.
tokenizer_config.json: 2.10kB [00:00, 12.3MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 1.40MB/s]
tokenizer.json: 1.80MB [00:00, 43.1MB/s]
special_tokens_map.json: 100% 414/414 [00:00<00:00, 3.88MB/s]
config.json: 100% 596/596 [00:00<00:00, 6.28MB/s]
2025-12-27 06:12:50.730043: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-27 06:12:50.747020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766815970.768264    3162 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory